In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
from pycocotools.coco import COCO
site = 'val'
img_dir = f'./data/jinan/pan/{site}/JPEGImages'
json_path = f'./data/jinan/fusion/{site}/{site}.json'

In [ ]:
coco = COCO(json_path)
img_ids = set(_['image_id'] for _ in coco.anns.values())

In [ ]:
from mmdet.apis import init_detector, show_result_pyplot, inference_detector
import mmcv
import glob
import os
work_dir = './checkpoints/tf_net'
config_file = glob.glob(os.path.join(work_dir, '*.py'))[0]
checkpoint_file = glob.glob(os.path.join(work_dir, '*.pth'))[0]
cfg_options = {}

model = init_detector(config_file, checkpoint_file, device='cuda:0', cfg_options=cfg_options)

In [ ]:
import numpy as np
import mmcv
import cv2
import torch
import tifffile as tiff
def show_mask_result(img, result, score_thr):
    # 基于掩码，提取轮廓显示
    bbox_result, segm_result = result[:2]
    bboxes = np.vstack(bbox_result)
    labels = [
        np.full(bbox.shape[0], i, dtype=np.int32)
        for i, bbox in enumerate(bbox_result)
    ]
    labels = np.concatenate(labels)
    # draw segmentation masks
    segms = None
    if segm_result is not None and len(labels) > 0:  # non empty
        segms = mmcv.concat_list(segm_result)
        if isinstance(segms[0], torch.Tensor):
            segms = torch.stack(segms, dim=0).detach().cpu().numpy()
        else:
            segms = np.stack(segms, axis=0)
    scores = bboxes[:, -1]
    inds = scores > score_thr
    bboxes = bboxes[inds, :]
    labels = labels[inds]
    if segms is not None:
        segms = segms[inds, ...]
    points = []
    for i in range(len(bboxes)):
        mask = segms[i].astype(np.uint8)
        contours, _ = cv2.findContours(mask[..., None], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(img, contours, -1, (0,0,255), 2)
        for contour in contours:
            points.append(len(contour))
    return img, sum(points) / len(points)

def show_polygon_result(img, result, score_thr):
    # 根据点结果，显示轮廓
    bbox_result, _, polygon_result = result

    bboxes = np.vstack(bbox_result)
    scores = bboxes[:, -1]
    inds = np.nonzero(scores > score_thr)[0]
    points = []
    for i in inds:
        poly = np.array(polygon_result[0][i]).reshape(-1, 2).astype(np.int32)
        points.append(len(poly))
        cv2.polylines(img, [poly], True, (0, 0, 255), 2)
    return img, sum(points) / len(points)

def show_det_result(img, result, score_thr):
    bbox_result = result
    bboxes = np.vstack(bbox_result)
    scores = bboxes[:, -1]
    inds = scores > score_thr
    bboxes = bboxes[inds, :]
    for box in bboxes:
        x1, y1, x2, y2 = box.astype(np.int32).tolist()[:4]
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    return img

def show_result(img_path, result, score_thr, ret_det=False):
    if os.path.splitext(img_path)[1].lower() == '.tif':
        img = tiff.imread(img_path)
        img = img[:, :, :3][:, :, ::-1]
        img = img.copy()
    else:
        img = cv2.imread(img_path)
    if ret_det:
        return show_det_result(img, result, score_thr=score_thr)
    if len(result) == 3:
        return show_polygon_result(img, result, score_thr=score_thr)
    else:
        return show_mask_result(img, result, score_thr=score_thr)

In [ ]:
print(len(img_ids))

In [ ]:
output_dir = './visual_tf_det'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
img_ids = list(img_ids)

for bi, img_id in enumerate(tqdm(img_ids)):
    fn = coco.load_imgs([img_id])[0]['file_name']
    img_path = os.path.join(img_dir, fn)
    
    result = inference_detector(model, img_path)
#     show_result_pyplot(model, img_path, result, score_thr=0.2)
    img_path = img_path.replace('pan', 'fusion')
    img = show_result(img_path, result, score_thr=0.5, ret_det=True)
    # img = img[:, :, ::-1]
    # plt.figure(figsize=(15, 10))
    # plt.axis('off')
    # plt.imshow(img)
    # plt.show()
    out_path = os.path.join(output_dir, fn)
    out_path = out_path.replace('.tif', '.png')
    cv2.imwrite(out_path, img)
    # if bi > 10:
    #     break